In [6]:
from bs4 import BeautifulSoup
import requests
import re
url = "https://www.antaranews.com/tag/cek-fakta/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

span_elements = soup.find_all('span', string=re.compile(r"9 Maret 2025"))

print(span_elements)


[<span class="text-dark text-capitalize">9 Maret 2025</span>]


In [8]:
hrefs = set()
titles = set()
for span in span_elements:
    # Find the nearest parent that contains an <a> child with href
    parent = span.find_parent()
    
    while parent:  # Keep going up in the hierarchy if necessary
        link = parent.find('a', href=True)
        if link:
            hrefs.add(link['href'])
            titles.add(link['title'])  # Store the found href
            break  # Stop once we find the first valid parent with a link
        parent = parent.find_parent()  # Move to the next level up

# Print all collected hrefs
for href in hrefs:
    print(href)

for title in titles:
    print(title)

https://www.antaranews.com/berita/4698137/libur-sekolah-lebih-awal-berikut-jadwal-lengkap-hari-libur-maret-2025
Libur sekolah lebih awal: Berikut jadwal lengkap hari libur Maret 2025


In [10]:
from bs4 import BeautifulSoup
import requests

# URL to scrape
url = "https://www.antaranews.com/berita/4698137/libur-sekolah-lebih-awal-berikut-jadwal-lengkap-hari-libur-maret-2025"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Dictionary to store parent elements and their <p> count
parent_p_count = {}

# Find all <p> elements
for p in soup.find_all('p'):
    parent = p.find_parent()  # Get the parent of <p>
    
    if parent:
        parent_p_count[parent] = parent_p_count.get(parent, 0) + 1  # Count <p> elements under each parent

# Find the parent with the most <p> children
if parent_p_count:
    max_parent = max(parent_p_count, key=parent_p_count.get)  # Get the parent with the max <p> count
    print(max_parent.get_text("\n", strip=True))  # Print only the text inside the parent
else:
    print("No parent with multiple <p> tags found.")


Jakarta (ANTARA) - Bulan Maret 2025 bertepatan dengan bulan suci Ramadhan 1446 Hijriah, di mana umat Muslim di seluruh dunia saat ini tengah menjalankan ibadah puasa. Pada saat tulisan ini dibuat, puasa telah berlangsung selama delapan hari sejak dimulai pada Sabtu, 1 Maret 2025.
Dalam beberapa hari ke depan, bulan Maret akan memasuki pertengahan, dan banyak masyarakat mulai mencari informasi mengenai jadwal libur, termasuk libur Lebaran.
Tidak hanya pekerja, jadwal libur ini juga menjadi perhatian bagi seluruh lapisan masyarakat, seperti pelajar, pegawai negeri, serta pelaku usaha yang perlu menyesuaikan agenda mereka selama Hari Raya Idul Fitri.
Selain itu, pada bulan Maret 2025, juga terdapat perayaan keagamaan lainnya, yakni Hari Raya Nyepi bagi umat Hindu serta perayaan Idul Fitri bagi umat Islam. Karena itu, terdapat beberapa hari libur nasional, cuti bersama, dan masa libur sekolah yang cukup panjang.
Berikut adalah rincian jadwal libur di bulan Maret 2025 berdasarkan peraturan 

In [1]:
import time

# Mapping months to Indonesian names
bulan = {
    "January": "Januari", "February": "Februari", "March": "Maret", "April": "April",
    "May": "Mei", "June": "Juni", "July": "Juli", "August": "Agustus",
    "September": "September", "October": "Oktober", "November": "November", "December": "Desember"
}

def current_date():
    raw_date = time.strftime("%d %B %Y", time.localtime())
    for eng, indo in bulan.items():
        raw_date = raw_date.replace(eng, indo)
    return raw_date

print(current_date())


12 Maret 2025


In [2]:
import requests
from bs4 import BeautifulSoup
import datetime

def generate_url():
    today = datetime.date.today()
    formatted_date = today.strftime("%m%%2F%d%%2F%Y")  # MM%2FDD%2FYYYY
    return f"https://news.detik.com/berita/indeks?date={formatted_date}"

def get_numbered_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the 'Prev' or 'Next' anchor tag
    prev_next_link = soup.find('a', string=lambda text: text and text.strip().lower() in ["prev", "next"])

    if prev_next_link:
        parent = prev_next_link.find_parent()  # Get the parent element
        if parent:
            numbered_links = set()  # Use a set to ensure no duplicates
            for anchor in parent.find_all('a', href=True):
                text = anchor.text.strip()
                if text.isdigit() and 1 <= int(text) <= 9:
                    if text.lower() not in ["prev", "next"]:  # Exclude prev and next
                        numbered_links.add(anchor['href'])  # Add to set to remove duplicates
            return list(numbered_links)  # Convert back to list before returning
    return []

# Example usage
url = generate_url()
numbered_links = get_numbered_links(url)
print(numbered_links)


['https://news.detik.com/berita/indeks?page=2&date=03/12/2025', 'https://news.detik.com/berita/indeks?page=3&date=03/12/2025', 'https://news.detik.com/berita/indeks?page=4&date=03/12/2025', 'https://news.detik.com/berita/indeks?page=1&date=03/12/2025']


In [ ]:
def get_articles(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    articles = {}  # Dictionary to store unique titles and links

    for h3 in soup.find_all('h3', class_='media__title'):
        a_tag = h3.find('a', class_='media__link', href=True)
        if a_tag:
            title = a_tag.get_text(strip=True)  # Remove extra spaces & newlines
            link = a_tag['href']

            # Store unique articles (avoiding duplicates)
            if link not in articles:
                articles[link] = title

    return articles


In [5]:
def extract_content(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all <p> tags within the div.detail__body-text
        content_div = soup.find('div', class_='detail__body-text')
        if content_div:
            paragraphs = content_div.find_all('p')
            paragraph_text = ' '.join(p.get_text(strip=True) for p in paragraphs)  # Join text without newlines
            return paragraph_text
        return ""  # Return empty string if no content found
    except requests.exceptions.RequestException as e:
        print(f"Error fetching content from {url}: {e}")
        return ""

In [ ]:
# Step 1: Get the main index page
url = generate_url()

# Step 2: Find pagination links
pagination_links = get_numbered_links(url)

# Step 3: Extract articles from the main page
articles = get_articles(url)

# Step 4: Scan each pagination page for more articles
for page_link in pagination_links:
    page_articles = get_articles(page_link)
    articles.update(page_articles)  # Merge, avoiding duplicates

# Step 5: Extract content for each article
data = []
for link, title in articles.items():
    print(f"Processing: {title}")  # Print progress
    content = extract_content(link)
    date = current_date()   # Placeholder for now, can extract if needed
    is_fake = 0  # Assuming articles are real by default
    data.append([title, link, date, content, is_fake])
    time.sleep(2)  # Avoid overwhelming the server

# Print final dataset
print("Data collection complete!")
for row in data:
    print(row)

Processing: TB Hasanuddin Minta Seskab Teddy Mundur dari TNI, Ungkit Penjelasan Istana
Processing: Bagaimana Persiapan Pengamanan Jalur Mudik? Yuk, Simak di Sini!
Processing: Jangan Lewatkan! #DemiIndonesia Bangkitkan Koperasi Hari Ini Hanya di detikcom
Processing: Ngaku Intel, Pria Minta 'Jatah Bensin' di Jakpus Ternyata Pecatan Polisi
Processing: Mendes Yandri Temui Jaksa Agung Adukan Dana Desa Dipakai Judol
Processing: IOJI Dorong Pemerintah Beri Hukuman Penjara Jika Pejabat Terlibat Pagar Laut
Processing: Kapolda Metro Tegakkan Disiplin, Pecat 4 Anggota Terbukti Melanggar
Processing: Barang-barang Ini Disita KPK dari Rumah Ridwan Kamil
Processing: Arus Balik, Menhub Ungkap Ada Usul WFA Juga Diberlakukan Usai Lebaran
Processing: Gubernur Banten-Kapolda Cek Gudang Bulog Pastikan Stok Beras Aman
Processing: Koster Ungkap Nama Ketut dan Nyoman Terancam Punah
Processing: Transmedia dan PAM JAYA Teken MoU Sosialisasi Proyek Penyediaan Air Minum
Processing: Penggunaan Bahu Jalan Tol Dalko

In [7]:
data

[['TB Hasanuddin Minta Seskab Teddy Mundur dari TNI, Ungkit Penjelasan Istana',
  'https://news.detik.com/berita/d-7819380/tb-hasanuddin-minta-seskab-teddy-mundur-dari-tni-ungkit-penjelasan-istana',
  '',
  'Anggota Komisi I DPR Fraksi PDIP TB Hasanuddin menilai SeskabLetkol Teddy Indra Wijayaseharusnya mundur dari TNI karena menduduki jabatan sipil. Hasanuddin menyinggung pernyataan Istana Kepresidenan soal jabatan Seskab berada di bawah Kementerian Sekretariat Negara, bukan Sekretariat Militer. Hasanuddin awalnya menyinggung daftar inventarisasi masalah (DIM) dari pemerintah ada 15 kementerian atau lembaga (K/L) yang bisa diisi oleh prajurit TNI aktif. Hasanuddin mengatakan sempat menyampaikan pendapat di Istana terkait posisi Teddy. "Saat itu saya menyarankan agar jika ingin mempertahankan status militer Mayor Teddy, maka posisinya sebaiknya ditempatkan di Sekretariat Militer. Ada beberapa jabatan di sana, seperti Kepala Biro Umum, Kepala Biro Tanda Pangkat, dan Kepala Biro Tanda Ja

In [9]:
def write_to_csv(data, filename):
    file_exists = os.path.isfile(filename)
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write header row if file doesn't exist
        if not file_exists:
            writer.writerow(['title', 'link', 'date', 'content', 'is_fake'])
        # Write data rows
        for row in data:
            writer.writerow(row)

In [ ]:
import os
import csv
# File name for the raw CSV
filename = 'dataset_detik_raw.csv'

# Write crawled data to CSV file
write_to_csv(data, filename)

print(f"Data has been written to {filename}")

Data has been written to dataset_detik_raw.csv


In [14]:
import re
def extract_title(text):
    # Normalize text to lowercase
    text = text.lower()

    # Define start phrases to find the actual title content
    start_phrases = [
        "\n"
    ]

    # Initialize the pattern with the start phrases
    start_pattern = "|".join(map(re.escape, start_phrases))
    pattern = f"({start_pattern})(.*)"

    # Use regular expressions to find content after the start phrases
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return match.group(2).strip()
    else:
        return text

# Open the CSV file in read mode to process titles
with open('dataset_detik_raw.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)

    # Skip the header row
    next(reader)

    # Create a list to store modified data
    modified_data = []

    # Iterate over each row in the CSV file
    for row in reader:
        title = row[0]
        link = row[1]
        date = row[2]
        content = row[3]
        is_fake = row[4]

        # Extract and normalize title
        extracted_title = extract_title(title)

        # Append modified data to the list
        modified_data.append([extracted_title, link, date, content, is_fake])

# File name for the final CSV
filename = 'dataset_detik.csv'

# Write modified data to CSV file
write_to_csv(modified_data, filename)

print(f"Data has been written to {filename}")


Data has been written to dataset_detik.csv


In [3]:
import datetime

In [8]:
from datetime import datetime  # Ensure you import datetime from the datetime module

def current_month_year():
    month_digit = datetime.now().strftime("%m")  # Get current month as two-digit string
    year_digit = datetime.now().strftime("%Y")  # Get current year as four-digit string
    return month_digit, year_digit

# Example usage:
month_digit, year_digit = current_month_year()
print(f"Current month: {month_digit}, Current year: {year_digit}")


Current month: 03, Current year: 2025


In [9]:
url = f'https://cekfakta.tempo.co/{year_digit}/{month_digit}'

In [10]:
print(url)

https://cekfakta.tempo.co/2025/03


In [9]:
import csv
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

def get_date():
    return datetime.today().strftime("%Y-%m-%d")

def get_tempo_links():
    url = f'https://cekfakta.tempo.co/{datetime.now().strftime("%Y/%m")}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links_data = []
    
    for article in soup.find_all('article', class_='text-card'):
        date_text = article.find('h4', class_='date')
        if date_text and 'jam' in date_text.get_text():
            a_tag = article.find('a', href=True)
            if a_tag:
                link = "https:" + a_tag['href'] if a_tag['href'].startswith("//") else a_tag['href']
                links_data.append((a_tag.get_text().strip(), link))
    
    return list(set(links_data))

# Example usage
tempo_links = get_tempo_links()
print(tempo_links)


[('Keliru: Video Richard Lee Promosikan Situs Judi Online Miliknya', 'https://cekfakta.tempo.co/fakta/3477/keliru-video-richard-lee-promosikan-situs-judi-online-miliknya'), ('Keliru: Video yang Diklaim Sebagai Tsunami', 'https://cekfakta.tempo.co/fakta/3476/keliru-video-yang-diklaim-sebagai-tsunami'), ('Keliru: Istri Presiden Prancis Emmanuel Macron adalah Ayahnya yang Transgender', 'https://cekfakta.tempo.co/fakta/3478/keliru-istri-presiden-prancis-emmanuel-macron-adalah-ayahnya-yang-transgender')]


In [13]:
def scrape_tempo_articles(tempo_links):
    data = []
    date = get_date()
    
    for title, link in tempo_links:
        category, title_news = title.split(':', 1) if ':' in title else (title, title)
        is_fake = 1 if category in ["Menyesatkan", "Keliru"] else 0
        
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        detail_in = soup.find('div', 'detail-in')
        p = detail_in.find_all('p')
        content = ' '.join(p.get_text() for p in p)
        
        data.append([title_news, link, date, content, is_fake])
    
    return data

scrape = scrape_tempo_articles(tempo_links)

In [3]:

def write_to_csv(data, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['title', 'link', 'date', 'content', 'is_fake'])
        writer.writerows(data)

def extract_content(text):
    match = re.search(r"(jakarta -|jakarta-)(.*)", text.lower(), re.DOTALL)
    return match.group(2).strip() if match else text

def process_raw(filename):
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        modified_data = [[row['title'], row['link'], row['date'], extract_content(row['content']), row['is_fake']] for row in reader]
    
    write_to_csv(modified_data, 'dataset_tempo.csv')
    print("Data has been written to dataset_tempo.csv")

In [4]:
filename = 'dataset_tempo_raw.csv'
tempo_links = get_tempo_links()

In [5]:
scraped = scrape_tempo_articles(tempo_links)

In [6]:
scraped

[['Video Richard Lee Promosikan Situs Judi Online Miliknya',
  'https://cekfakta.tempo.co/fakta/3477/keliru-video-richard-lee-promosikan-situs-judi-online-miliknya',
  '2025-03-13',
  'SEBUAH video beredar di Facebook [arsip] yang memperlihatkan dokter kecantikan, Richard Lee, mempromosikan situs judi online (judol) miliknya. Video yang diunggah tanggal 9 Maret 2025 itu memperlihatkan Richard Lee berbaju putih menyampaikan ingin berbagi atau bersedekah melalui situs yang bernama Ayu89. Mereka yang bermain di situs Ayu89, dia akan memberikan jackpot senilai Rp3-7 juta rupiah. Keinginan tersebut diamini oleh pendakwah Felix Siauw dan Derry Sulaiman.  Namun, benarkah Richard Lee mempromosikan situs judol miliknya? Hasil cek fakta Tempo menunjukkan, video saat Richard Lee berbicara tentang judi online adalah hasil rekayasa teknologi kecerdasan buatan. Tempo memverifikasi video itu menggunakan Google Lens, penelusuran YouTube, serta alat deteksi deepfake. Video versi asli yang memperlihatka

In [7]:
write_to_csv(scraped, filename)

In [8]:
process_raw(filename)

Data has been written to dataset_tempo.csv


In [14]:
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import re

yesterday = (datetime.today() - timedelta(days=1)).strftime('%d/%m/%Y')
print(yesterday)


12/03/2025


In [25]:
base_url = 'https://www.kompas.com/cekfakta/hoaks-atau-fakta'
today = datetime.today().strftime('%d/%m/%Y')
data = []
page = 1

url = f"{base_url}/{page}" if page > 1 else base_url

In [26]:
url

'https://www.kompas.com/cekfakta/hoaks-atau-fakta'

In [27]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

news_items = soup.find_all('a', class_='cekfakta-list-link')

In [28]:
news_items

[<a class="cekfakta-list-link" href="https://www.kompas.com/cekfakta/read/2025/03/12/153800182/-hoaks-link-untuk-pencairan-thr-rp-2-75-juta-dari-pemerintah">
 <p class="cekfakta-text-date">12/03/2025, 15:38 WIB</p>
 <h1 class="cekfakta-list-title">[HOAKS] Link untuk Pencairan THR Rp 2,75 Juta dari Pemerintah</h1>
 </a>,
 <a class="cekfakta-list-link" href="https://www.kompas.com/cekfakta/read/2025/03/12/142800482/-klarifikasi-video-robot-optimus-lipat-baju-dinarasikan-keliru">
 <p class="cekfakta-text-date">12/03/2025, 14:28 WIB</p>
 <h1 class="cekfakta-list-title">[KLARIFIKASI] Video Robot Optimus Lipat Baju Dinarasikan Keliru</h1>
 </a>,
 <a class="cekfakta-list-link" href="https://www.kompas.com/cekfakta/read/2025/03/12/123523182/infografik-mobil-dirusak-massa-karena-pelaku-tabrak-lari-bukan-tidak">
 <p class="cekfakta-text-date">12/03/2025, 12:35 WIB</p>
 <h1 class="cekfakta-list-title">INFOGRAFIK: Mobil Dirusak Massa karena Pelaku Tabrak Lari, Bukan Tidak Bayar Bensin</h1>
 </a>,


In [37]:
data = []

In [38]:
yesterday

'12/03/2025'

In [39]:
def extract_content(text):
    text = text.lower()
    start_phrases = ["kompas.com -", ".kompas.com", "kompas.com-"]
    start_pattern = "|".join(map(re.escape, start_phrases))
    pattern = f"({start_pattern})(.*)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(2).strip() if match else text

In [42]:
for item in news_items:
    date_elem = item.find('p', class_='cekfakta-text-date')
    date_str = date_elem.get_text(strip=True).split(',')[0]
    print(date_str)
    if date_str != yesterday:
        break  # Stop if we find an older date
    
    found_today = True
    title = item.find('h1').get_text(strip=True)
    print(title)
    link = item['href']

    content_response = requests.get(link)
    content_soup = BeautifulSoup(content_response.text, 'html.parser')
    content_paragraphs = content_soup.find('div', class_='read__content').find_all('p')
    content = ' '.join(p.get_text(strip=True) for p in content_paragraphs)
    extracted_content = extract_content(content)
    
    data.append([title, link, date_str, extracted_content, 0])

12/03/2025
[HOAKS] Link untuk Pencairan THR Rp 2,75 Juta dari Pemerintah
12/03/2025
[KLARIFIKASI] Video Robot Optimus Lipat Baju Dinarasikan Keliru
12/03/2025
INFOGRAFIK: Mobil Dirusak Massa karena Pelaku Tabrak Lari, Bukan Tidak Bayar Bensin
12/03/2025
[KLARIFIKASI] Foto Kate Middleton Hamil Anak Keempat adalah Manipulasi AI
12/03/2025
[HOAKS] Video Prabowo Umumkan Hukuman Mati untuk Koruptor
12/03/2025
[KLARIFIKASI] Kasus Antam Rugikan Negara Rp 3,3 Triliun, Bukan Rp 5,9 Kuadriliun
11/03/2025


In [43]:
data

[['[HOAKS] Link untuk Pencairan THR Rp 2,75 Juta dari Pemerintah',
  'https://www.kompas.com/cekfakta/read/2025/03/12/153800182/-hoaks-link-untuk-pencairan-thr-rp-2-75-juta-dari-pemerintah',
  '12/03/2025',
  'tunjangan hari raya (thr) dinantikan setiap pekerja menjelang lebaran. hal ini menjadikan informasi mengenai thr menjadi perhatian masyarakat. beredarlinkatau tautan di media sosial yang diklaim sebagai akses pencairan thr dari pemerintah sebesar rp 2,75 juta per kartu keluarga (kk). klaim yang disampaikan, thr tersebut diberikan kepada seluruh masyarakat indonesia. namun setelah ditelusuri tim cek faktakompas.com,tautan yang disebarkan merupakanphishing. informasi mengenai pencairan thr sebesar rp 2,75 juta dari pemerintah disebarkan oleh akun facebookinidanini. berikut narasi yang ditulis salah satu akun pada rabu (12/3/2025): info bantuan thr 2025 silahkan dafta sekarang cara daftar silahkan klik link yang sudah di sediakan akun facebooktangkapan layar konten hoaks di sebuah a